In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


import os
import pickle
from datetime import datetime


from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report,
    precision_recall_curve, average_precision_score
)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline


RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


os.makedirs('data', exist_ok=True)
os.makedirs('outputs', exist_ok=True)
os.makedirs('models', exist_ok=True)


In [6]:



# Check if file exists
data_path = 'data/cleaned_data.csv'


df = pd.read_csv(data_path)




df.head()

,NACCID,NACCADC,PACKET,FORMVER,VISITMO,VISITDAY,VISITYR,NACCVNUM,NACCAVST,NACCNVST,NACCDAYS,NACCFDYS,BIRTHMO,BIRTHYR,NACCAGE,NACCAGEB,SEX,HISPANIC,HISPOR,HISPORX,RACE,RACEX,RACESEC,RACESECX,RACETER,RACETERX,NACCNIHR,PRIMLANG,PRIMLANX,EDUC,MARISTAT,NACCLIVS,INDEPEND,RESIDENC,HANDED,NACCREAS,NACCREFR,INBIRMO,INBIRYR,INSEX,NEWINF,INHISP,INHISPOR,INHISPOX,INRACE,INRACEX,INRASEC,INRASECX,INRATER,INRATERX,INEDUC,INRELTO,INRELTOX,INKNOWN,INLIVWTH,NACCFAM,NACCMOM,NACCFADM,NACCAM,NACCAMX,NACCAMS,NACCAMSX,NACCFFTD,NACCFM,NACCFMX,NACCFMS,NACCFMSX,NACCBMI,NACCUDSD,DEMENTED
0,NACC002909,186,I,3.0,12,28,2022,1,2,2,391.0,0.0,5,1952,70.0,70.0,1,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,16.0,1,4,1,1,2,7,2,5.0,1962.0,2.0,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,16.0,1.0,NaN,8.0,1.0,1.0,0.0,0,9.0,NaN,9.0,NaN,0,9.0,NaN,9.0,NaN,32.4,3,0
1,NACC002909,186,F,3.0,1,23,2024,2,2,2,391.0,391.0,5,1952,71.0,70.0,1,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,16.0,1,2,1,1,2,7,2,5.0,1962.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,0.0,0,9.0,NaN,9.0,NaN,0,9.0,NaN,9.0,NaN,30.7,3,0
2,NACC003487,186,I,3.0,11,15,2023,1,1,1,0.0,0.0,12,1956,66.0,66.0,1,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,16.0,1,2,1,1,2,7,2,3.0,1958.0,2.0,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,16.0,1.0,NaN,40.0,1.0,0.0,0.0,0,9.0,NaN,9.0,NaN,0,9.0,NaN,9.0,NaN,23.7,1,0
3,NACC004352,186,I,3.0,10,5,2021,1,1,1,0.0,0.0,1,1958,63.0,63.0,2,1,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,2,NaN,16.0,1,2,2,1,2,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,888.8,4,1
4,NACC004687,186,I,3.0,11,14,2022,1,1,1,0.0,0.0,2,1945,77.0,77.0,1,1,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1,NaN,12.0,3,1,1,1,2,1,2,2.0,1950.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,12.0,3.0,NaN,NaN,0.0,9.0,0.0,0,9.0,NaN,9.0,NaN,0,9.0,NaN,9.0,NaN,19.0,1,0


In [13]:


print("🔍 Data Inspection\n")



print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]:,} columns")


print(f"\nData Types:")
print(df.dtypes.value_counts())

missing = df.isnull().sum().sum()
print(f"\nMissing Values: {missing:,} ({missing/(df.shape[0]*df.shape[1])*100:.2f}%)")


print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("="*70)

🔍 Data Inspection

Shape: 195,196 rows x 70 columns

Data Types:
float64    31
int64      23
object     16
Name: count, dtype: int64

Missing Values: 4,832,118 (35.36%)

Memory Usage: 189.92 MB


📊 Splitting data into train and test sets...



NameError: name 'X' is not defined